<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/colab/Analyse_poverty_mobility_2019_NY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Spark SQL Documentation:** 
https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

In [1]:
!pip install ipython-autotime

%load_ext autotime

time: 3.64 ms (started: 2021-04-06 19:30:50 +00:00)


In [2]:
# Install required dependancies
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 71kB/s 
     |████████████████████████████████| 204kB 32.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=8f45561f4d606dcbead823072fd82d8d53517112c40945d927e984f4cba82062
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 30 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [3]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

time: 121 ms (started: 2021-04-06 19:38:40 +00:00)


In [4]:
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

time: 260 ms (started: 2021-04-06 19:38:47 +00:00)


In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

time: 5.19 s (started: 2021-04-06 19:38:52 +00:00)


In [6]:
# Mount drive with data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 28.7 s (started: 2021-04-06 19:39:01 +00:00)


In [7]:
# Demographic Data NY
# !ls drive/MyDrive/big-data-project/data/clean-data/ny/ny.csv

# !ls drive/MyDrive/big-data-project/data/clean-data/ny/social/2019/


time: 1.38 ms (started: 2021-04-06 19:39:38 +00:00)


In [8]:
# Set path to data folder
path = Path('drive/MyDrive/big-data-project/data/clean-data')
city = 'ny'

time: 1.38 ms (started: 2021-04-06 19:39:53 +00:00)


In [9]:
# Read data into RDD
rdd_soc = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'social/2019/*.csv'))
rdd_soc.show()

+---+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+--------------------+------------------------+--------------------------------+
|_c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell_t

In [11]:
# View schema
rdd_soc.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- cbg: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- date_range_end: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- distance_traveled_from_home: string (nullable = true)
 |-- bucketed_distance_traveled: string (nullable = true)
 |-- median_dwell_at_bucketed_distance_traveled: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- median_home_dwell_time: string (nullable = true)
 |-- bucketed_home_dwell_time: string (nullable = true)
 |-- at_home_by_each_hour: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)
 |-- destination_cbgs: string (nullable = true)
 |-- delivery_behavior_devices: string (nullable = true)
 |-- median_non_home_dwell_time: string (nullable = true)
 |-- candidate_device_count: string (nullable = true)
 |-- bucketed_away_from_home_ti

In [12]:
# # Take small sample of data to experiment with
# rdd_soc = rdd_soc.limit(100)
# rdd_soc.show()

time: 1.33 ms (started: 2021-04-06 19:41:23 +00:00)


In [14]:
rdd_soc.createOrReplaceTempView('clean_ny')
rdd_soc = spark.sql('SELECT cbg, date_range_start, device_count, completely_home_device_count, part_time_work_behavior_devices, full_time_work_behavior_devices FROM clean_ny')
rdd_soc.show()

+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+
|         cbg|    date_range_start|device_count|completely_home_device_count|part_time_work_behavior_devices|full_time_work_behavior_devices|
+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+
|360050177022|2019-12-27T00:00:...|          41|                           6|                              1|                              1|
|360050248002|2019-12-27T00:00:...|          51|                          14|                              1|                              1|
|360050363003|2019-12-27T00:00:...|          75|                          33|                              4|                              1|
|360470084003|2019-12-27T00:00:...|          46|                           9|                              3|                              4|
|36047

In [15]:
# View schema
rdd_soc.printSchema()

root
 |-- cbg: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)

time: 6.72 ms (started: 2021-04-06 19:44:06 +00:00)


In [23]:
# Add completely home percentage column
rdd_soc = rdd_soc.withColumn('completely_home_percentage', (rdd_soc['completely_home_device_count']/rdd_soc['device_count']) * 100)
rdd_soc = rdd_soc.withColumn('part_time_work_percentage', (rdd_soc['part_time_work_behavior_devices']/rdd_soc['device_count']) * 100)
rdd_soc = rdd_soc.withColumn('full_time_work_percentage', (rdd_soc['full_time_work_behavior_devices']/rdd_soc['device_count']) * 100)

rdd_soc.show()

+---+----------------+------------+----------------------------+-------------------------------+-------------------------------+--------------------------+-------------------------+-------------------------+
|cbg|date_range_start|device_count|completely_home_device_count|part_time_work_behavior_devices|full_time_work_behavior_devices|completely_home_percentage|part_time_work_percentage|full_time_work_percentage|
+---+----------------+------------+----------------------------+-------------------------------+-------------------------------+--------------------------+-------------------------+-------------------------+
+---+----------------+------------+----------------------------+-------------------------------+-------------------------------+--------------------------+-------------------------+-------------------------+

time: 3.16 s (started: 2021-04-06 19:50:19 +00:00)


In [18]:
rdd_soc.printSchema()

root
 |-- cbg: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)
 |-- completely_home_percentage: double (nullable = true)
 |-- part_time_work_percentage: double (nullable = true)
 |-- full_time_work_percentage: double (nullable = true)

time: 2.77 ms (started: 2021-04-06 19:46:17 +00:00)


In [20]:
# create temp view
rdd_soc.createOrReplaceTempView('new_clean_ny')

time: 30.7 ms (started: 2021-04-06 19:46:40 +00:00)


In [21]:
# get mean percentage for each cbg
rdd_soc_ny = spark.sql('SELECT cbg, AVG(completely_home_percentage) AS mean_completely_home_percentage FROM new_clean_ny GROUP BY cbg')
rdd_soc_ny.createOrReplaceTempView('mean_completely_home')
rdd_soc_ny.show()

+---+-------------------------------+
|cbg|mean_completely_home_percentage|
+---+-------------------------------+
+---+-------------------------------+

time: 7.13 s (started: 2021-04-06 19:47:35 +00:00)


In [22]:
# get mean percentage for full time work for each cbg
rdd_soc_ft = spark.sql('SELECT cbg, AVG(full_time_work_percentage) AS mean_full_time_work_percentage FROM new_clean_ny GROUP BY cbg')
rdd_soc_ft.createOrReplaceTempView('mean_fulltime_work')
rdd_soc_ft.show()

+---+------------------------------+
|cbg|mean_full_time_work_percentage|
+---+------------------------------+
+---+------------------------------+

time: 4.75 s (started: 2021-04-06 19:48:19 +00:00)
